In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from datetime import datetime
import statsmodels.api as sm


In [4]:
df_metdata=pd.read_json(r'/content/drive/MyDrive/P_F_H/1.json',lines=True)
df_metdata

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274996,Maven Realty,"Maven Realty, 2180 McMillan Ave #71672, North ...",0x88fe64e7daceaaab:0x101c046dcadcc9c3,None,32.857737,-79.985839,[Real estate agency],4.9,15,None,"[[Saturday, Open 24 hours], [Sunday, Open 24 h...",None,Open 24 hours,"[0x88fe657160ccc965:0xc675c849dc6d96ba, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
274997,Stone Castle Fence,"Stone Castle Fence, 124 Cypress Gardens Rd, Mo...",0x88fe59f4602c95fd:0x57e2b1904d9e6949,None,33.089951,-80.078021,"[Fence contractor, Construction company, Contr...",4.0,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",None,Closed ⋅ Opens 7AM Mon,"[0x88fe8d1444299bbb:0x91daa2e82c59ea94, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
274998,Health Insurance Solutions Team,"Health Insurance Solutions Team, 6650 Rivers A...",0x88fe71f9617c15e1:0x89518c1d251706bf,None,32.918858,-80.027952,[Health insurance agency],5.0,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...","{'Service options': ['Online appointments'], '...",Closed ⋅ Opens 8AM Mon,"[0x88fe6fe45b8d3f15:0x41d4b940a40b246a, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
274999,Bernard Building Center,"Bernard Building Center, 27300 M-32, M-32, Hil...",0x4d34c7517d6e6e29:0x7fe553ee2f090cf0,None,45.056691,-83.894423,"[Building materials store, Electrical supply s...",4.7,25,None,"[[Saturday, 8AM–3PM], [Sunday, Closed], [Monda...","{'Service options': ['Curbside pickup', 'Deliv...",Closed ⋅ Opens 7:30AM Mon,"[0x4d335f6408d395d9:0xc20dea80d4c100d8, 0x4d34...",https://www.google.com/maps/place//data=!4m2!3...


In [5]:
df_metdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state             195523 non-null  object 
 13  relative_results  238771 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


In [6]:
df_metdata.category

0                                                [Pharmacy]
1                                        [Textile exporter]
2                                       [Korean restaurant]
3                                            [Fabric store]
4                                            [Fabric store]
                                ...                        
274996                                 [Real estate agency]
274997    [Fence contractor, Construction company, Contr...
274998                            [Health insurance agency]
274999    [Building materials store, Electrical supply s...
275000                                   [Auto repair shop]
Name: category, Length: 275001, dtype: object

In [7]:
df_metdata.name.value_counts()

United States Postal Service                   728
Redbox                                         609
U-Haul Neighborhood Dealer                     467
7-Eleven                                       458
Shell                                          399
                                              ... 
Destiny Arts Center                              1
Amazon Hub Counter (Harrison - Whole Foods)      1
Center of Attention Hair Salon                   1
La Clínica Dental                                1
HMR Motor Works and Auto Care                    1
Name: name, Length: 219822, dtype: int64

In [9]:
def categorias_estados( categoria: str ):
 cant=0
 for i  in range(len(df_metdata.category)):

     if type(df_metdata.category[i])==list:
         for j in df_metdata.category[i]:
             if j==categoria:
                 cant=cant+1
     else:
         if df_metdata.category[i]==categoria:
             cant=cant+1

 if cant==0:
     print('Ingreso incorrecto,recuerde escribir todo en minusculas')
 return{'En el estado elegido hay una cantidad':cant,'de la categoria':categoria}

In [20]:
df_metdata[df_metdata['name']=="McDonald's"]

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
5012,McDonald's,"McDonald's, 1205 S Main St, Manteca, CA 95336",0x80904101ce001281:0x76db23c5d22346ae,"Classic, long-running fast-food chain known fo...",37.785995,-121.218062,"[Fast food restaurant, Breakfast restaurant, C...",2.4,48,$,None,"{'Service options': ['Takeout', 'Delivery'], '...",None,"[0x8090410018dc2657:0xed7a807ae3eeee6d, 0x8090...",https://www.google.com/maps/place//data=!4m2!3...
6679,McDonald's,"McDonald's, 1000 Palisades Center Dr, West Nya...",0x89c2e9cf8e139235:0x24bfb20e9e09f260,"Classic, long-running fast-food chain known fo...",41.097768,-73.955392,"[Fast food restaurant, Breakfast restaurant, C...",2.2,18,$,None,"{'Service options': ['Delivery'], 'Highlights'...",None,"[0x89c2e9e6ef010ddb:0xe923f7207b70d6f9, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
6690,McDonald's,"McDonald's, 341 5th Ave, New York, NY 10016",0x89c259a9b55adb77:0xfe5e87207e736efc,"Classic, long-running fast-food chain known fo...",40.747916,-73.984586,[Fast food restaurant],3.1,16,None,None,{'Service options': ['Delivery']},None,"[0x89c259a9b2e6f0b1:0xca9f9eef13b45d33, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
7564,McDonald's,"McDonald's, 2400 Aviation Dr, Dallas, TX 75261",0x864c2b8770fc957d:0xdbc6f271dec0dcef,"Classic, long-running fast-food chain known fo...",32.902380,-97.037369,[Fast food restaurant],4.3,4,$,None,"{'Service options': ['Curbside pickup', 'Takeo...",None,None,https://www.google.com/maps/place//data=!4m2!3...
16258,McDonald's,"McDonald's, 571 Walton Blvd, Las Cruces, NM 88001",0x86de3d67b2b54843:0xa54f3893ef44d96,"Classic, long-running fast-food chain known fo...",32.315042,-106.750119,[Fast food restaurant],4.0,5,$,"[[Wednesday, 5AM–11PM], [Thursday, 5AM–11PM], ...","{'Service options': ['Delivery'], 'Dining opti...",Permanently closed,"[0x86de17e200933271:0xb244ae1ca1025934, 0x86de...",https://www.google.com/maps/place//data=!4m2!3...
31585,McDonald's,"McDonald's, 1205 S Main St, Manteca, CA 95336",0x80904101ce001281:0x76db23c5d22346ae,"Classic, long-running fast-food chain known fo...",37.785995,-121.218062,"[Fast food restaurant, Breakfast restaurant, C...",2.4,48,$,None,"{'Service options': ['Takeout', 'Delivery'], '...",None,"[0x8090410018dc2657:0xed7a807ae3eeee6d, 0x8090...",https://www.google.com/maps/place//data=!4m2!3...
33252,McDonald's,"McDonald's, 1000 Palisades Center Dr, West Nya...",0x89c2e9cf8e139235:0x24bfb20e9e09f260,"Classic, long-running fast-food chain known fo...",41.097768,-73.955392,"[Fast food restaurant, Breakfast restaurant, C...",2.2,18,$,None,"{'Service options': ['Delivery'], 'Highlights'...",None,"[0x89c2e9e6ef010ddb:0xe923f7207b70d6f9, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
33263,McDonald's,"McDonald's, 341 5th Ave, New York, NY 10016",0x89c259a9b55adb77:0xfe5e87207e736efc,"Classic, long-running fast-food chain known fo...",40.747916,-73.984586,[Fast food restaurant],3.1,16,None,None,{'Service options': ['Delivery']},None,"[0x89c259a9b2e6f0b1:0xca9f9eef13b45d33, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
34137,McDonald's,"McDonald's, 2400 Aviation Dr, Dallas, TX 75261",0x864c2b8770fc957d:0xdbc6f271dec0dcef,"Classic, long-running fast-food chain known fo...",32.902380,-97.037369,[Fast food restaurant],4.3,4,$,None,"{'Service options': ['Curbside pickup', 'Takeo...",None,None,https://www.google.com/maps/place//data=!4m2!3...
42831,McDonald's,"McDonald's, 571 Walton Blvd, Las Cruces, NM 88001",0x86de3d67b2b54843:0xa54f3893ef44d96,"Classic, long-running fast-food chain known fo...",32.315042,-106.750119,[Fast food restaurant],4.0,5,$,"[[Wednesday, 5AM–11PM], [Thursday, 5AM–11PM], ...","{'Service options': ['Delivery'], 'Dining opti...",Permanently closed,"[0x86de17e200933271:0xb244ae1ca1025934, 0x86de...",https://www.google.com/maps/place//data=!4m2!3...


In [19]:
df_metdata[df_metdata['name']=="McDonald's"].category

5012      [Fast food restaurant, Breakfast restaurant, C...
6679      [Fast food restaurant, Breakfast restaurant, C...
6690                                 [Fast food restaurant]
7564                                 [Fast food restaurant]
16258                                [Fast food restaurant]
31585     [Fast food restaurant, Breakfast restaurant, C...
33252     [Fast food restaurant, Breakfast restaurant, C...
33263                                [Fast food restaurant]
34137                                [Fast food restaurant]
42831                                [Fast food restaurant]
64600     [Fast food restaurant, Breakfast restaurant, H...
69334     [Fast food restaurant, Breakfast restaurant, C...
105835    [Fast food restaurant, Breakfast restaurant, C...
108416    [Fast food restaurant, Breakfast restaurant, C...
128968    [Fast food restaurant, Breakfast restaurant, C...
137130    [Fast food restaurant, Hamburger restaurant, R...
140017                               [Fa

In [21]:
df_metdata[df_metdata['name']=="McDonald's"].category.count()

29

In [24]:
df_metdata.iloc[5012]

name                                                       McDonald's
address                 McDonald's, 1205 S Main St, Manteca, CA 95336
gmap_id                         0x80904101ce001281:0x76db23c5d22346ae
description         Classic, long-running fast-food chain known fo...
latitude                                                    37.785995
longitude                                                 -121.218062
category            [Fast food restaurant, Breakfast restaurant, C...
avg_rating                                                        2.4
num_of_reviews                                                     48
price                                                               $
hours                                                            None
MISC                {'Service options': ['Takeout', 'Delivery'], '...
state                                                            None
relative_results    [0x8090410018dc2657:0xed7a807ae3eeee6d, 0x8090...
url                 

In [23]:
df_metdata.iloc[5012].category

['Fast food restaurant',
 'Breakfast restaurant',
 'Coffee shop',
 'Hamburger restaurant',
 'Restaurant',
 'Sandwich shop']

In [ ]:
df_metdata.url[0]


'https://www.google.com/maps/place//data=!4m2!3m1!1s0x88f16e41928ff687:0x883dad4fd048e8f8?authuser=-1&hl=en&gl=us'

**Realizo una busqueda en los datos usando terminos en la aplicacion Yelp usados tambien:**

In [10]:
categorias_estados('Pharmacy')

{'En el estado elegido hay una cantidad': 2225, 'de la categoria': 'Pharmacy'}

In [11]:
categorias_estados('Food')

{'En el estado elegido hay una cantidad': 36, 'de la categoria': 'Food'}

In [12]:
categorias_estados('Fast Food')

Ingreso incorrecto,recuerde escribir todo en minusculas


{'En el estado elegido hay una cantidad': 0, 'de la categoria': 'Fast Food'}

In [ ]:
categorias_estados('Cafeteria')

{'En el estado elegido hay una cantidad': 46, 'de la categoria': 'Cafeteria'}

**Muestro los nombres de los locales encontrados**

In [ ]:

def categorias_estados_l( categoria: str ):
 cant=0
 for i  in range(len(df_metdata.category)):

     if type(df_metdata.category[i])==list:
         for j in df_metdata.category[i]:
             if j==categoria:
                 cant=cant+1
                 print('nombre',df_metdata.name[i])
     else:
         if df_metdata.category[i]==categoria:
             cant=cant+1
             print('nombre',df_metdata.name[i])
 if cant==0:
     print('Ingreso incorrecto,recuerde escribir todo en minusculas')

 return{'En el estado elegido hay una cantidad':cant,'de categoria':categoria}

In [ ]:
categorias_estados_l('Food')

nombre Alaska General Seafoods
nombre Valbert Drive In Feed & Farm
nombre La Jolla Food Distributors LLC
nombre Lian Hwa Foods Inc
nombre Alaska General Seafoods
nombre Valbert Drive In Feed & Farm
nombre La Jolla Food Distributors LLC
nombre Lian Hwa Foods Inc
nombre California Sweet Potato Growers
nombre Eggman Family Honey
nombre Global Produce Sales
nombre Atlas Walnuts LLC
nombre Super Botanica Del Sol
nombre Orlando Foods Inc
nombre Southeast-Atlantic Beverage
nombre Prima Sausage Co
nombre Sterling Hill Potatoes, remote storage
nombre Brewsky's Bottle Shop
nombre Caballero Grill
nombre Maximo Boatyard
nombre Fireside Winery in the Village
nombre Paradise Koi Ponds
nombre Best Bargain 99 Cent
nombre Alabama Railcar Services
nombre Trask Fish Hatchery
nombre Jack & Jill Nut Shop
nombre Wholesale USA
nombre Reinhart Food Services
nombre Wisconsin Distributors
nombre Jackson Oven Supply
nombre Illini Institutional Foods Inc
nombre Shoals Flight Center
nombre End Of The Vine Produce


{'En el estado elegido hay una cantidad': 36, 'de categoria': 'Food'}

In [ ]:
categorias_estados_l('Cafeteria')

nombre KHANA KHAZANA "A Traditional Indian Cafe"
nombre Chefness Bakery
nombre Sandburg Café
nombre Cuatro Gato
nombre MJ CAFE
nombre EL DOLLAR MONEY STATION #2
nombre KHANA KHAZANA "A Traditional Indian Cafe"
nombre Chefness Bakery
nombre Sandburg Café
nombre Cuatro Gato
nombre MJ CAFE
nombre EL DOLLAR MONEY STATION #2
nombre Nordstrom Ebar Artisan Coffee
nombre Green Leaf's & Bananas
nombre DOÑA CONCHA MEAT MARKET
nombre Uz's Donuts
nombre Papa Bambinos Pizza Pasta
nombre Charlotte Cafe
nombre IKEA Restaurant
nombre Los Angelitos
nombre Luby's
nombre McCormick Dining Hall
nombre Cafe Gallery & Market
nombre Georgia Pines Dining Facility
nombre Green Leaf's & Bananas
nombre Charles White Dining Hall
nombre Epicurean Feast
nombre The Piazza Cafetaria
nombre 2HELA2
nombre Fair Oaks Express Cafe
nombre Mezza Latin House
nombre Patrick J McAtee Dining Center
nombre North Servery
nombre Empanadas and coffee
nombre Stern Dining
nombre Marketplace Cafe
nombre Microsoft Cafe 26
nombre Fort My

{'En el estado elegido hay una cantidad': 46, 'de categoria': 'Cafeteria'}

In [ ]:
df_metdata2=pd.read_json(r'/content/drive/MyDrive/P_F_H/2.json',lines=True)
df_metdata2

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.127472e+20,Benjamin Vidot,1560405775843,5,Hot and cold sandwich's cold sodas and much mo...,None,None,0x89c2f48ae322c115:0xd28226c7ffc52291
1,1.048890e+20,Joel Bing,1558663754858,4,Nice environment cozy good food,None,None,0x89c2f48ae322c115:0xd28226c7ffc52291
2,1.063806e+20,Danny Ramos,1558467764714,5,Not good.,None,None,0x89c2f48ae322c115:0xd28226c7ffc52291
3,1.169573e+20,TRACY MITCHELL,1556481687751,3,Decent,None,None,0x89c2f48ae322c115:0xd28226c7ffc52291
4,1.084914e+20,Geovanni Rojas,1453077455490,1,(Translated by Google) Very unfortunate the ra...,None,None,0x89c2f48ae322c115:0xd28226c7ffc52291
...,...,...,...,...,...,...,...,...
149995,1.085485e+20,Komgrit Chukiert,1576891719351,4,None,None,None,0x89d9417bb8ab4d57:0x39a39051c7aa452f
149996,1.052827e+20,Widad IB,1517095258821,5,None,None,None,0x89d9417bb8ab4d57:0x39a39051c7aa452f
149997,1.137627e+20,John Wiswall,1549132248712,4,None,None,None,0x89d9417bb8ab4d57:0x39a39051c7aa452f
149998,1.145119e+20,Alex,1562050481023,5,None,None,None,0x89d9417bb8ab4d57:0x39a39051c7aa452f


In [ ]:
df_metdata3=pd.read_json(r'/content/drive/MyDrive/P_F_H/3.json',lines=True)
df_metdata3

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.121598e+20,Kevin Noble,1531531850612,4,Large electrical wholesaler with very good sto...,None,None,0x89d9f3c896fd648f:0x26e65f259859802f
1,1.120503e+20,Joseph Hanover,1471217722578,4,Good service and they know what those talking ...,None,None,0x89d9f3c896fd648f:0x26e65f259859802f
2,1.055255e+20,morice lee,1604339382504,4,Best,None,None,0x89d9f3c896fd648f:0x26e65f259859802f
3,1.112953e+20,Joe Milano,1544139014031,5,Excellant service and atmosphere,None,None,0x89d9f3c896fd648f:0x26e65f259859802f
4,1.037596e+20,Joe R,1552236266839,5,Has it all,None,None,0x89d9f3c896fd648f:0x26e65f259859802f
...,...,...,...,...,...,...,...,...
149995,1.081464e+20,Mya Rhodus,1531316168670,5,Real authentic back home food I absolutely lov...,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
149996,1.173153e+20,Juan Lopez,1521869879997,5,Great place great service friendly and respect...,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
149997,1.122765e+20,zin ski,1565202265454,5,"Excellent food, price and service!!!!",None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
149998,1.042520e+20,Lorrianne Williams,1550577745823,5,Great guyanese food everyday.,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5


In [ ]:
df_metdata4=pd.read_json(r'/content/drive/MyDrive/P_F_H/4.json',lines=True)
df_metdata4

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.121496e+20,Annette Green,1570882095310,4,The food is excellent,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
1,1.027794e+20,Yonette Hinds,1552593653693,5,Great tasting food.,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
2,1.167207e+20,Dennies G,1531254304902,5,Great food,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
3,1.149411e+20,Slimthick24 W,1544780492780,5,"Good, delicious food.",None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
4,1.070571e+20,Nicola Hunte,1543697846703,4,Food very tasty,None,None,0x89c25b60682deed3:0x3e007cd09ec0f0e5
...,...,...,...,...,...,...,...,...
149995,1.058475e+20,Alanna Dolan,1534076526753,5,None,None,None,0x89c2c8dbefd938b5:0x9a069980fc47245
149996,1.050053e+20,Sixto Sinche,1620097082758,5,Very nice place food looks great taste great a...,None,None,0x89c25daf13bc726b:0x7f609addad46f073
149997,1.109932e+20,Ahmed Ali,1623284057992,5,David was very polite and professional\nWill d...,None,None,0x89c25daf13bc726b:0x7f609addad46f073
149998,1.023495e+20,I AM Crystal,1615329430120,5,Food was Amazing. Large portions family meal f...,None,None,0x89c25daf13bc726b:0x7f609addad46f073
